In [1]:
import safe

/Users/chenwuhao/Dev/md4/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
encoded_safe = safe.encode("COc1cc2c(c(OC)c1OC)-c1ccc(NCCCC(=O)N3CCC(N=C(O)OC(C)(C)C)CC3)c(=O)cc1C(N=C(C)O)CC2")


In [15]:
tokenizer = safe.SAFETokenizer.from_pretrained("datamol-io/safe-gpt")
tokenizer = tokenizer.get_pretrained()

/Users/chenwuhao/Dev/md4/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [19]:
encoded_safe = safe.encode("CC(C)(O)c1coc(S(=O)(=O)NC(O)=Nc2c3c(cc4c2CCC4)CCC3)c1")
tokenized = tokenizer.convert_ids_to_tokens(tokenizer.encode(encoded_safe))
print(f"SAFE: {encoded_safe}")
print(f"Tokenized: {tokenized}")

SAFE: N5C(O)=Nc1c2c(cc3c1CCC3)CCC2.c16coc(S5(=O)=O)c1.CC6(C)O
Tokenized: ['[CLS]', 'N', '5', 'C', '(', 'O', ')', '=', 'N', 'c', '1', 'c', '2', 'c', '(', 'c', 'c', '3', 'c', '1', 'C', 'C', 'C', '3', ')', 'C', 'C', 'C', '2', '.', 'c', '1', '6', 'c', 'o', 'c', '(', 'S', '5', '(', '=', 'O', ')', '=', 'O', ')', 'c', '1', '.', 'C', 'C', '6', '(', 'C', ')', 'O', '[SEP]']


In [21]:
print(len(tokenized))

57
